In [2]:
import numpy as np


### Formula para invertir matriz
Vamos a necesitar invertir matrices 3x3 para esto necesitamos el determinante

In [4]:
def det(A):
    return A[0,0]*A[1,1]*A[2,2]+A[0,1]*A[1,2]*A[2,0]+A[0,2]*A[1,0]*A[2,1]-A[0,2]*A[1,1]*A[2,0]-A[0,1]*A[1,0]*A[2,2]-A[0,0]*A[1,2]*A[2,1]
def inv(A):
    return (1/det(A))*np.array([[A[1,1]*A[2,2]-A[2,1]*A[1,2],A[0,2]*A[2,1]-A[2,2]*A[0,1],A[0,1]*A[1,2]-A[1,1]*A[0,2]],[A[1,2]*A[2,0]-A[2,2]*A[1,0],A[0,0]*A[2,2]-A[2,0]*A[0,2] ,A[0,2]*A[1,0]-A[1,2]*A[0,0]], [A[1,0]*A[2,1]-A[2,0]*A[1,1] ,A[0,1]*A[2,0]-A[2,1]*A[0,0] ,A[0,0]*A[1,1]-A[1,0]*A[0,1]]])


# Metodo de Newton
Para el caso unidimensional al tomar un $u_0$ arbitrario podemos definir $u_n$
$$u_1 = u_0 -\frac{f(u_0)}{f'(u_0)}$$
lo cual converge a algun $z$ raiz de la funcion. 

Para el caso de varias variables, digamos si queremos resolver el sistema
$$f(x,y,z)=0  $$
$$g(x,y,z)= 0 $$
$$h(x,y,z)=0 $$
tomamos $F=(f,g,h)$ y dado un $v_0$ y $J_{v_n}$ es el Jacobiano en $v_n$ inductivamente tenemos:
$$v_{n+1} = v_n-J_{v_0}^{-1}F(v_0) $$
lo cual converge a algun $0$ cercano a $v_0$

In [5]:
def f(v):
    return (1/3)*v[0]**3+v[1]
def g(v):
    return v[2]**2-v[1]
def h(v):
    return v[1]**2-v[0]*v[1]
def F(v):
    return np.array([f(v),g(v),h(v)])
def fx(x,y,z):
    return x**2
def fy(x,y,z):
    return 1
def fz(x,y,z):
    return 0
def gx(x,y,z):
    return -1
def gy(x,y,z):
    return 0
def gz(x,y,z):
    return 2*z
def hx(x,y,z):
    return -y
def hy(x,y,z):
    return 2*y-x
def hz(x,y,z):
    return 0
def Jac(v):
    return np.array([[fx(v[0],v[1],v[2]),fy(v[0],v[1],v[2]),fz(v[0],v[1],v[2])],[gx(v[0],v[1],v[2]),gy(v[0],v[1],v[2]),gz(v[0],v[1],v[2])],[hx(v[0],v[1],v[2]),hy(v[0],v[1],v[2]),hz(v[0],v[1],v[2])]])



In [6]:
def Newton(v0, eps, ITERMAX):
    
    contador =0
    inf=np.amax(v0)
    v0 = np.array(v0,dtype='float64')
    while eps < inf and contador <= ITERMAX:
        v1=v0-np.dot(inv(Jac(v0)),F(v0))
        contador = contador+1
        inf=np.amax(np.abs(v1-v0))
        v0=v1
       
    if eps < inf:
        return 'Despues de '+str(ITERMAX)+' pasos el algoritmo no converge'
        
    return v1, contador


# Metodo de la secante 
El metodo de newton asume que conocemos una expresion analitica para cada derivada parcial, la cual podemos aproximar con la siguiente formula:
$$J_{i,j}=\frac{\partial F_i(x_1,...x_n)}{\partial x_i} \approx \frac{f_i(x_1^n,...,x_j^{n+1},...x_n^n)-f_i(x_1^n,...,x_n^n)}{x_j^{n+1}-x_j^n}=\hat J_{i,j}^{n+1}$$

In [7]:
def approxJac(v1,v2):
    a11=(f([v2[0],v1[1],v1[2]])-f(v1))/(v2[0]-v1[0])
    a12=(f([v1[0],v2[1],v1[2]])-f(v1))/(v2[1]-v1[1])
    a13=(f([v1[0],v1[1],v2[2]])-f(v1))/(v2[2]-v1[2])
    a21=(g([v2[0],v1[1],v1[2]])-g(v1))/(v2[0]-v1[0])
    a22=(g([v1[0],v2[1],v1[2]])-g(v1))/(v2[1]-v1[1])
    a23=(g([v1[0],v1[1],v2[2]])-g(v1))/(v2[2]-v1[2])
    a31=(h([v2[0],v1[1],v2[2]])-h(v1))/(v2[0]-v1[0])
    a32=(h([v1[0],v2[1],v1[2]])-h(v1))/(v2[1]-v1[1])
    a33=(h([v1[0],v1[1],v2[2]])-h(v1))/(v2[2]-v1[2])
    
    return np.array([[a11,a12,a13],[a21,a22,a23],[a31,a32,a33]])


Una vez obtenida esta aproximacion de la secante obtenemos el algoritmo donde  dados $v_0$ y $v_1$

$$v_{n+1}=v_n- \left(\hat J_{i,j}^{n+1}\right)^{-1} f(v_n) $$

In [8]:
def Secante(v0, v1, eps, ITERMAX):
    
    v0=np.array(v0,dtype='float64')
    v1=np.array(v1,dtype='float64')
    inf=np.amax(np.abs(v1-v0))
    contador = 0
    while eps < inf and contador <= ITERMAX:
        J=approxJac(v0,v1)
        v2=v1-np.dot(inv(J),F(v1))
        v0=v1
        v1=v2
        inf=np.amax(np.abs(v1-v0))
        contador = contador+1
    if eps < inf:
        return 'Despues de '+str(ITERMAX)+' pasos el algoritmo no converge'

    return v1,contador



# Calculos finales

In [12]:

#el primer argumento es el vector inicial de newton. el segundo argumento es epsilon, y el tercer argumento es ITERMAX
print(Newton([0.1,200,0.3],0.01,250))
#el primer argumento son los dos vectores iniciales. el segundo argumento es epsilon, y el tercer argumento es ITERMAX
print(Secante([1,2,3],[5,6,8],eps=0.01,ITERMAX=400))

(array([-3.44951864e-03,  2.62221296e-13,  4.26203632e-02]), 28)
(array([-3.53443505e-02,  6.96665152e-06,  3.97576315e-03]), 16)
